In [59]:
from Adafruit_IO import Client, Feed
from i2c_sensor_classes import Ky015Sensor
import time

In [9]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
base = BaseOverlay("base.bit")

In [10]:
KY015_ADDR = 8
SCL_PIN = 2
SDA_PIN = 3

TEMP_FEED_GROUP = 'temperature-measurements'
HUM_FEED_GROUP = 'humidity-measurements'
BRIGHT_FEED_GROUP = 'brightness-measurements'

liba = MicroblazeLibrary(base.PMODA, ['i2c'])
ky015 = Ky015Sensor(liba, SDA_PIN, SCL_PIN, KY015_ADDR)

UNAME = input('Enter username: ')
KEY = input('Enter Key: ')

TEMP_FEED = 'temperature'
HUM_FEED = 'humididty'

#Create a client that can send information to the Adafruit Cloud
aio = Client(UNAME, KEY)
#create the feed that the data gets sent to
temp_feed = aio.feeds(TEMP_FEED)
hum_feed = aio.feeds(HUM_FEED)

response = input('\t-Read Temp(y/n): ')
while response != 'n':
    temp, hum = ky015.get_temp_and_hum_data()
    #send the data to desired feed
    print('\t\tTemp: {}C, Hum: {}%'.format(temp,hum))
    aio.send(hum_feed.key, hum)
    aio.send(temp_feed.key, temp)
    response = input('\t-Read Temp(y/n): ')

Enter username: 
Enter Key: 


RequestError: Adafruit IO request failed: 404 Not Found - not found - that is an invalid URL, please check the API documentation at https://io.adafruit.com/api/docs to make sure your URL is correct

In [52]:
#class that defines the sdl and scl pins for an i2c master
class I2CMaster:
    def __init__(self, sdl:int, scl:int):
        self.sdl_pin = sdl
        self.scl_pin = scl
        print(global_var)
# #class for a sensor that can send data to the cloud       
class AioDevice:
    def __init__(self, aio:Client, sensor):
        self.aio = aio
        self.sensor = sensor
        self.sensor_data = None
    #override these functions in children
    #get the sensor_data
    def get_data(self):
        pass
    def publish_data(self):
        pass

class TempHumSensor(AioDevice):
    def __init__(self, aio:Client, sensor:Ky015Sensor):
        super().__init__(aio,sensor)
    def get_data(self):
        temp, hum = self.sensor.get_temp_and_hum_data()
        self.sensor_data = (temp, hum)
        return self.sensor_data
    def publish_data(self, temp_feed, hum_feed):
        temp, hum = self.sensor_data
        aio.send(temp_feed.key, temp)
        aio.send(hum_feed.key, hum)
             
class BrightnessSensor(AioDevice):
    def __init__(self, aio, i2c_master):
        pass

In [53]:
class HabitatMonitor():
    def __init__(self, aio, client_id):
        print('creating new HabitatMonitor...')
        self.CLIENT_ID_FEED = 'client-ids'
        self.FEED_GROUPS = {
            'temperature':'temperature-measurements',  
            'humidity':'humidity-measurements', 
            'brightness':'brightness-measurements' 
        }
        self.aio = aio
        self.client_id = client_id
        self.feeds = {}
        self.client_number = None
        sucess = self._initilize_feeds()
        if not(sucess):
            print("FAILURE!")
            pass
        self.temp_and_hum_sensor = self._init_ky015_sensor()
        self.brightness_sensor = self._init_ky018_sensor()

    #public methods
    def set_feeds(self):
        self.feeds['temperature'] = self.aio.feeds('temperature-measurements.temperature-{}-{}'.format(self.client_id, self.client_number))
        self.feeds['humidity'] = self.aio.feeds('humidity-measurements.humidity-{}-{}'.format(self.client_id, self.client_number))
        self.feeds['brightness'] = self.aio.feeds('brightness-measurements.brightness-{}-{}'.format(self.client_id, self.client_number))

#     def read_and_pubilsh_all_sensor_data(self):
#         self.read_all_data()
#         self.publish_all_data()
#     def publish_all_data(self):
# #         self.brightness_sensor.publish_data(sefl.feeds['brightness'])
    def read_data(self):
        temp, hum = self.temp_and_hum_sensor.get_data()
#         bright = self.brightness_sensor.get_data()
        return temp, hum
    def publish_data(self):
        self.temp_and_hum_sensor.publish_data(self.feeds['temperature'], self.feeds['humidity'])
           
    #private methods
    def _initilize_feeds(self):
        if not(self._is_new_client()):
            client_id_feed = self.aio.feeds(self.CLIENT_ID_FEED)
            self.client_number = len(self.aio.data(client_id_feed.key))
            self.set_feeds()
            return True
        else:
            if self._create_new_feeds():
                return True
            return False
    def _is_new_client(self):
        client_ids_feed = self.aio.feeds(self.CLIENT_ID_FEED)
        client_ids = self.aio.data(client_ids_feed.key)
        for cid in client_ids:
            if self.client_id == cid.value:
                return False
        return True
    def _can_create_feeds(self):
        #TODO send request to add a new client to the cloud
        #the server board can recevie the request and 
        #decide if the board with this client_id is allowed to join
        return True #for now always return True
    def _create_new_feeds(self):
        client_id_feed = self.aio.feeds(self.CLIENT_ID_FEED)
        cur_client_count = len(self.aio.data(client_id_feed.key))
        if not(self._can_create_feeds()):
            return False
        self.client_number = cur_client_count + 1
        aio.send(client_id_feed.key, self.client_id)
        for feed_name, group_name in self.FEED_GROUPS.items():
            feed = Feed(name='{}-{}-{}'.format(feed_name, self.client_id, self.client_number,))
            aio.create_feed(feed, group_name)
            self.feeds[feed_name] = feed
        return True
    def _init_ky015_sensor(self):
        slave_address = 8
        scl_pin = 2
        sda_pin = 3
        liba = MicroblazeLibrary(base.PMODA, ['i2c'])
        ky015 = Ky015Sensor(liba, sda_pin, scl_pin, slave_address)
        return TempHumSensor(self.aio, ky015)
    def _init_ky018_sensor(self):
        pass #TODO
    

In [54]:
UNAME = input('Enter username: ')
KEY = input('Enter Key: ')
aio = Client(UNAME, KEY)
client_id = 'joeys-pynq-board'
# print(aio.feeds())
# print('-------------')
# print(aio.feeds('client-ids'))
hab_mon = HabitatMonitor(aio, client_id) 

Enter username: vsaah
Enter Key: aio_Souf951aUwD276YjXUSdxMFqT34a
creating new HabitatMonitor...


In [64]:
print('Getting Habitat Measurements...')
tt = time.time()
temp, hum = hab_mon.read_data()
mt = round((time.time()-tt)*1000, 2)
print('\tTemp: {}, Hum: {}\n\tMeasuring took {} ms'.format(temp, time, mt))
print('Pushing Data To the Cloud...')
tt = time.time()
hab_mon.publish_data()
mt = round(time.time()-tt, 2)
print('\tPushing took {} s'.format(mt))

Getting Habitat Measurements...
	Temp: 20.0, Hum: <module 'time' (built-in)>
	Measuring took 10.01 ms
Pushing Data To the Cloud...
	Pushing took 0.72 s


In [66]:
run_time = 1*60*60 #1 hour
publish_period = 10*60 #10 minutes
start_time = time.time()
dt = 0
while dt<run_time:
    print('Getting Habitat Measurements...')
    tt = time.time()
    temp, hum = hab_mon.read_data()
    mt = round((time.time()-tt)*1000, 2)
    print('\tTemp: {}, Hum: {}\n\tMeasuring took {} ms'.format(temp, time, mt))
    print('Pushing Data To the Cloud...')
    tt = time.time()
    hab_mon.publish_data()
    mt = round(time.time()-tt, 2)
    print('\tPushing took {} s'.format(mt))
    time.sleep(publish_period)
    dt = time.time()-start_time

Getting Habitat Measurements...
	Temp: 20.0, Hum: <module 'time' (built-in)>
	Measuring took 10.05 ms
Pushing Data To the Cloud...
	Pushing took 0.77 s
Getting Habitat Measurements...
	Temp: 20.08, Hum: <module 'time' (built-in)>
	Measuring took 8.72 ms
Pushing Data To the Cloud...
	Pushing took 0.74 s
Getting Habitat Measurements...
	Temp: 20.08, Hum: <module 'time' (built-in)>
	Measuring took 6.9 ms
Pushing Data To the Cloud...
	Pushing took 0.74 s
Getting Habitat Measurements...
	Temp: 20.08, Hum: <module 'time' (built-in)>
	Measuring took 8.8 ms
Pushing Data To the Cloud...
	Pushing took 0.77 s
Getting Habitat Measurements...
	Temp: 20.08, Hum: <module 'time' (built-in)>
	Measuring took 8.81 ms
Pushing Data To the Cloud...
	Pushing took 0.74 s


KeyboardInterrupt: 